**TODO**

* figure out how to use RigidBodyDynamics

In [1]:
using RigidBodyDynamics
using StaticArrays

In [18]:
g = -9.81 # gravitational acceleration in z-direction
world = RigidBody{Float64}("world")
twolinkarm = Mechanism(world; gravity = SVector(0, 0, g))

Spanning tree:
Vertex: world (root)
No non-tree joints.

In [20]:
axis = SVector(0., 0., 1.) # joint axis
I_1 = 0.333 # moment of inertia about joint axis
c_1 = -0.5 # center of mass location with respect to joint axis
m_1 = 1. # mass
frame1 = CartesianFrame3D("upper_link") # the reference frame in which the spatial inertia will be expressed
inertia1 = SpatialInertia(frame1, I_1 * axis * axis.', m_1 * SVector(0, 0, c_1), m_1)
upperlink = RigidBody(inertia1)
shoulder = Joint("shoulder", Revolute(axis))
beforeShoulderToWorld = eye(Transform3D, frame_before(shoulder), default_frame(world))
attach!(twolinkarm, world, shoulder, beforeShoulderToWorld, upperlink)

Spanning tree:
Vertex: world (root)
  Vertex: upper_link, Edge: shoulder
No non-tree joints.

In [21]:
l_1 = -1. # length of the upper link
I_2 = 0.333 # moment of inertia about joint axis
c_2 = -0.5 # center of mass location with respect to joint axis
m_2 = 1. # mass
inertia2 = SpatialInertia(CartesianFrame3D("lower_link"), I_2 * axis * axis.', m_2 * SVector(0, 0, c_2), m_2)
lowerlink = RigidBody(inertia2)
elbow = Joint("elbow", Revolute(axis))
beforeElbowToAfterShoulder = Transform3D(frame_before(elbow), shoulder.frameAfter, SVector(0, 0, l_1))
attach!(twolinkarm, upperlink, elbow, beforeElbowToAfterShoulder, lowerlink)

Spanning tree:
Vertex: world (root)
  Vertex: upper_link, Edge: shoulder
    Vertex: lower_link, Edge: elbow
No non-tree joints.

In [69]:
state = MechanismState{Float64}(twolinkarm)
configuration(state, shoulder)[:] = 0.0
configuration(state, elbow)[:] = 0.0
velocity(state, shoulder)[:] = 0.0
velocity(state, elbow)[:] = 0.0;

setdirty!(state)

2-element Array{Float64,1}:
 0.0
 0.0

In [16]:
ts, qs, vs = simulate(state, 0.1, Δt = 1e-2);

In [64]:
num_velocities(twolinkarm)
tau = Vector{Float64}(num_velocities(twolinkarm))
result = DynamicsResult(Float64, twolinkarm)

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] RigidBodyDynamics.DynamicsResult(::Type{Float64}, ::RigidBodyDynamics.Mechanism{Float64}) at ./deprecated.jl:57
 [3] include_string(::String, ::String) at ./loading.jl:515
 [4] include_string(::Module, ::String, ::String) at /Users/Kazu/.julia/v0.6/Compat/src/Compat.jl:577
 [5] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /Users/Kazu/.julia/v0.6/IJulia/src/execute_request.jl:154
 [6] eventloop(::ZMQ.Socket) at /Users/Kazu/.julia/v0.6/IJulia/src/eventloop.jl:8
 [7] (::IJulia.##14#17)() at ./task.jl:335
while loading In[64], in expression starting on line 3


RigidBodyDynamics.DynamicsResult{Float64,Float64}(Spanning tree:
Vertex: world (root)
  Vertex: upper_link, Edge: shoulder
    Vertex: lower_link, Edge: elbow
No non-tree joints., [2.26153e-314 2.25836e-314; 2.25836e-314 2.25843e-314], [0.0, 0.0], Array{Float64}(0,2), Float64[], [0.0, 0.0], Float64[], Float64[], RigidBodyDynamics.UnsafeFastDict(RigidBody: "world"=>Wrench expressed in "world":
angular: [0.0, 0.0, 0.0], linear: [0.0, 0.0, 0.0],RigidBody: "upper_link"=>Wrench expressed in "world":
angular: [0.0, 0.0, 0.0], linear: [0.0, 0.0, 0.0],RigidBody: "lower_link"=>Wrench expressed in "world":
angular: [0.0, 0.0, 0.0], linear: [0.0, 0.0, 0.0]), RigidBodyDynamics.UnsafeFastDict(RigidBody: "world"=>Wrench expressed in "world":
angular: [0.0, 0.0, 0.0], linear: [0.0, 0.0, 0.0],RigidBody: "upper_link"=>Wrench expressed in "world":
angular: [0.0, 0.0, 0.0], linear: [0.0, 0.0, 0.0],RigidBody: "lower_link"=>Wrench expressed in "world":
angular: [0.0, 0.0, 0.0], linear: [0.0, 0.0, 0.0]), Ri

In [107]:
function controller(state)
    # TODO add calculations of required tau
    tau1 = [1., 0.]
end

function controlled_dynamics!(vd::AbstractArray, sd::AbstractArray, t, state)
    tau = controller(state)
    dynamics!(result, state, tau)
    copy!(vd, result.v̇)
    copy!(sd, result.ṡ) 
end

const dyn! = controlled_dynamics!
# const dyn! = damped_dynamics!
# sink = DrakeVisualizerSink(vis)
const sink = ExpandingStorage{Float64}(10000)
const integrator = MuntheKaasIntegrator(dyn!, runge_kutta_4(Float64), sink);
# integrate(integrator, state, 10., 1e-4, maxRealtimeRate = Inf)

In [100]:
integrate(integrator, state, 1e-4, 1e-4, maxRealtimeRate = Inf)

In [101]:
sink.qs

18-element Array{Array{Float64,1},1}:
 [0.0, 0.0]               
 [1.5015e-8, -1.5015e-8]  
 [6.00601e-8, -6.00601e-8]
 [6.00601e-8, -6.00601e-8]
 [1.35135e-7, -1.35135e-7]
 [2.4024e-7, -2.4024e-7]  
 [3.75375e-7, -3.75375e-7]
 [5.40541e-7, -5.40541e-7]
 [7.35736e-7, -7.35736e-7]
 [9.60961e-7, -9.60961e-7]
 [1.21622e-6, -1.21622e-6]
 [1.5015e-6, -1.5015e-6]  
 [1.81682e-6, -1.81682e-6]
 [2.16216e-6, -2.16216e-6]
 [2.16216e-6, -2.16216e-6]
 [2.53754e-6, -2.53754e-6]
 [2.53754e-6, -2.53754e-6]
 [2.94294e-6, -2.94294e-6]

In [79]:
stages = integrator.stages
q0, v0, s0 = stages.q0, stages.vstep, stages.sstep

(Float64[], Float64[], Float64[])

In [86]:
q0 = configuration(state)

2-element Array{Float64,1}:
 0.0
 0.0

In [81]:
function plot_two_link_arm(state)
    # TODO double-check two link lengths
    # Use animate?
    
end

0-element Array{Float64,1}